In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
lstm = nn.LSTM(3, 3) # input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, 3))) for _ in range(5)] # output sequence of 5

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(
    torch.randn((1, 1, 3))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out, 'out')
print(hidden, 'hidden')

Variable containing:
(0 ,.,.) = 
 -0.2664  0.2560  0.1826

(1 ,.,.) = 
 -0.5134  0.0751 -0.1857

(2 ,.,.) = 
 -0.1719 -0.0072 -0.2800

(3 ,.,.) = 
  0.0155  0.0457 -0.3457

(4 ,.,.) = 
  0.1337  0.1122 -0.3655
[torch.FloatTensor of size 5x1x3]
 out
(Variable containing:
(0 ,.,.) = 
  0.1337  0.1122 -0.3655
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.2382  0.1585 -0.6873
[torch.FloatTensor of size 1x1x3]
) hidden


In [5]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [7]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(inputs)
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5]

Variable containing:
-1.1004 -1.0800 -1.1158
-1.0913 -1.0838 -1.1211
-1.1898 -0.9966 -1.1191
-1.1842 -1.0083 -1.1112
-1.1659 -0.9728 -1.1701
[torch.FloatTensor of size 5x3]



In [13]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

for epoch in range(1):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        
        print('sentence:\n', sentence)
        print('tags:\n', tags)
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        print('model.hidden:\n', model.hidden)
        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        print('sentence_in:\n', sentence_in)
        targets = prepare_sequence(tags, tag_to_ix)
        print('targets:\n', sentence_in)
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()


Variable containing:
-0.8552 -1.1356 -1.3722
-0.8430 -1.2335 -1.2790
-0.9254 -1.2135 -1.1827
-0.8036 -1.3973 -1.1874
-0.7373 -1.4735 -1.2294
[torch.FloatTensor of size 5x3]

sentence:
 ['The', 'dog', 'ate', 'the', 'apple']
tags:
 ['DET', 'NN', 'V', 'DET', 'NN']
model.hidden:
 (Variable containing:
(0 ,.,.) = 
  0  0  0  0  0  0
[torch.FloatTensor of size 1x1x6]
, Variable containing:
(0 ,.,.) = 
  0  0  0  0  0  0
[torch.FloatTensor of size 1x1x6]
)
sentence_in:
 Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5]

targets:
 Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5]

sentence:
 ['Everybody', 'read', 'that', 'book']
tags:
 ['NN', 'V', 'DET', 'NN']
model.hidden:
 (Variable containing:
(0 ,.,.) = 
  0  0  0  0  0  0
[torch.FloatTensor of size 1x1x6]
, Variable containing:
(0 ,.,.) = 
  0  0  0  0  0  0
[torch.FloatTensor of size 1x1x6]
)
sentence_in:
 Variable containing:
 5
 6
 7
 8
[torch.LongTensor of size 4]

targets:
 Variable containing:
 5
 6
 7


In [12]:

# See what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores)

Variable containing:
-1.0953 -1.0266 -1.1798
-0.9713 -1.1002 -1.2426
-1.1216 -0.9847 -1.2018
-1.3090 -0.9073 -1.1199
-1.2002 -1.0219 -1.0818
[torch.FloatTensor of size 5x3]

Variable containing:
-0.3375 -1.3118 -4.0695
-3.0587 -0.0558 -4.9151
-2.6603 -3.4162 -0.1084
-0.0279 -4.2398 -4.3313
-3.9039 -0.0298 -4.6842
[torch.FloatTensor of size 5x3]

